In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import pandas as pd
import numpy as n
import os
import json
import re

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
path = '/content/drive/My Drive/Data/gtdnorm.csv'
df = pd.read_csv(path)

In [ ]:
# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
for i in df.iloc[0]:
  if not (i >= 0 and i <= 1):
    print(i)

10.0


In [ ]:
from torch.utils.data import Dataset

In [ ]:
df = df.iloc[0:93500]

In [ ]:
class GTD(Dataset):
    def __init__(self, df):

        self.x = torch.tensor(df[[i for i in df.columns if i != 'target']].values, dtype=torch.float32)
        self.y = torch.tensor(df.target.values, dtype=torch.long)
        self.n_samples = len(df)

    def __getitem__(self, index):
        x = self.x[index]
        y = self.y[index]
        return x, y

    def __len__(self):
        return self.n_samples

In [ ]:
gtd_ds = GTD(df)

In [ ]:
len(gtd_ds)

93500

In [ ]:
from torch.utils.data import random_split, DataLoader

train_size = int(0.8 * len(df))
test_size = len(df) - train_size
train_dataset, test_dataset = random_split(gtd_ds, [train_size, test_size])

In [ ]:
batch_size = 100
num_iters = 3000
input_dim = df.shape[1]-1 # num_features = 43
num_hidden = 100 # num of hidden nodes
output_dim = len(df.target.unique()) #11

learning_rate = 0.001  # More power so we can learn faster! previously it was 0.001

In [ ]:
num_epochs = 25 #num_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [ ]:
for i in train_loader:
  print(i)
  break

[tensor([[0.5197, 0.5361, 0.8195,  ..., 1.0000, 0.0000, 1.0000],
        [0.3525, 0.5571, 0.4635,  ..., 0.9000, 0.0000, 0.9000],
        [0.5521, 0.5140, 0.5372,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.5038, 0.5683, 0.6424,  ..., 1.0000, 0.0000, 1.0000],
        [0.4118, 0.4863, 0.5988,  ..., 0.0000, 0.0000, 0.0000],
        [0.5774, 0.4703, 0.4973,  ..., 0.0000, 0.0000, 0.0000]]), tensor([ 4,  9, 10,  4,  0, 10, 10,  0,  0, 10,  0,  6, 10,  3,  5, 10,  0,  0,
         0,  0,  6,  0,  0,  0,  0,  0,  0,  0,  3,  6,  0,  0,  0,  0, 10,  4,
         0, 10,  0, 10,  0, 10,  0, 10,  0,  0,  0, 10, 10,  0,  1, 10,  0, 10,
         0,  0, 10,  0, 10,  0,  0, 10,  2,  0,  0,  0,  1,  0,  0,  1,  0,  0,
        10,  1,  1,  2,  0, 10,  1,  0, 10,  0, 10,  5,  0,  0, 10,  0,  0,  0,
         0,  5, 10,  6,  0,  0,  0,  2, 10,  0])]


In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)


        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        out, _ = self.gru(x, (h0.detach()))

        # Index hidden state of last time step
        out = self.fc(self.relu(out[:, -1, :]))

        return out

# Define hyperparameters
input_size = input_dim  # Number of input features
hidden_size = 64  # Number of LSTM units
num_layers = 2  # Number of LSTM layers
output_size = 11  # Number of output classes

# Instantiate the model
model = LSTMModel(input_size, hidden_size, num_layers, output_size).to(device)
print(model)

LSTMModel(
  (gru): GRU(1579, 64, num_layers=2, batch_first=True)
  (fc): Linear(in_features=64, out_features=11, bias=True)
  (relu): ReLU()
)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
num_epochs = 1000

In [ ]:
x = inputs.reshape(100, 1, input_dim)

NameError: ignored

In [ ]:
x.shape

In [ ]:
iter = 0
for epoch in range(num_epochs):

  for i, (inputs, labels) in enumerate(train_loader):
      inputs = inputs.to(device)
      labels = labels.to(device)

      optimizer.zero_grad()

      outputs = model(inputs.reshape(100, 1, input_dim))

      loss = criterion(outputs, labels)

      loss.backward()

      optimizer.step()

      iter += 1

  print('Epoch [{}/{}] Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [ ]:
import numpy as np

In [ ]:
yp = np.array([])
yt = np.array([])

total = 0
correct = 0

for input, labels in test_loader:

                input = input.to(device)

                # Forward pass only to get logits/output
                outputs = model(input.reshape(100, 1, input_dim))

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs, 1)


                yp = np.concatenate((yp, predicted.numpy()))
                yt = np.concatenate((yt, labels.numpy()))

                # Total number of labels
                total += labels.size(0)


                # Total correct predictions
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                    correct += (predicted == labels).sum()

print(yt.shape, yp.shape)
print(yt[:10])
print(yp[:10])


In [ ]:
accuracy = correct / total * 100
accuracy

In [ ]:
res = yt==yp
print(res.sum())

In [ ]:
from sklearn import metrics

cm = metrics.confusion_matrix(yt, yp)

In [ ]:
cm

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(9, 9))
sns.heatmap(cm, annot = True, fmt = '0.3f', linewidth = 0.5, square = True, cbar = False)
plt.ylabel('Actual Values')
plt.xlabel('Predicted Values')
plt.show

In [ ]:
print(metrics.classification_report(yt, yp))